## Keans算法
- Kmeans是无监督的聚类算法
- Kmeans的中心思想：
  事先确定常数K，常数K意味着最终的聚类类别数，首先随机选定初始点为质心，并通过计算每一个样本与质心之间的相似度(这里为欧式距离)，将样本点归到最相似的类中，接着，重新计算每个类的质心(即为类中心)，重复这样的过程，直到质心不再改变，最终就确定了每个样本所属的类别以及每个类的质心。由于每次都要计算所有的样本与每一个质心之间的相似度，故在大规模的数据集上，K-Means算法的收敛速度比较慢。

#### Kmeans伪代码：
1)选择K个点作为初始质心  
2)repeat  
    将每个点指派到最近的质心，形成K个簇  
    重新计算每个簇的质心  
3)until 簇不发生变化或达到最大迭代次数

#### 算法步骤：
- 1.选择聚类的个数k（kmeans算法传递超参数的时候，只需设置最大的K值）
- 2.任意产生k个聚类，然后确定聚类中心，或者直接生成k个中心。
- 3.对每个点确定其聚类中心点。
- 4.再计算其聚类新中心。
- 5.重复以上步骤直到满足收敛要求。（通常就是确定的中心点不再改变。）
<img src="./images/Kmeans.png" style="width:800px;height:270px;float:left">

#### Kmeans常用距离
- 欧式距离
- 曼哈顿距离
- 应用:
a.去除孤立点，离群点，只针对度量算法，解决方法（常用归一化预处理方法 ）
b.离散化

#### Kmeans的优缺点
##### 优点
- 1、原理比较简单，实现也是很容易，收敛速度快。
- 2、聚类效果较优。
- 3、算法的可解释度比较强。
- 4、主要需要调参的参数仅仅是簇数k。
#### 缺点
- 1、K值需要预先给定，K值的选取不好把握
- 2、K-Means算法对初始选取的聚类中心点是敏感的
- 3、对离群点， 噪声敏感 （中心点易偏移）
- 4、如果各隐含类别的数据不平衡，比如各隐含类别的数据量严重失衡，或者各隐含类别的方差不同，则聚类效果不佳
- 5、结果不一定是全局最优，只能保证局部最优（与K的个数及初值选取有关）
- 6、对于不是凸的数据集比较难收敛

### Kmeans算法优化
四种硬聚类算法(K-means++,二分K-means,ISODATA和Kernel K-means)

#### K-means++：
那么初始点怎么选呢？
- 第一个簇心A随机找，是因为一开始你不知道哪个是簇心；
- 第二个簇心B要找距离A最远的，是因为簇心之间要相距远一些，如果很近的话，很容易当作一类，影响聚类效果；
- 第三个簇心C也是同样的，它得离A、B远一些；其它依次类推。
<img src="./images/Kmeans++.png" style="width:800px;height:270px;float:left">

#### ISODATA：
- 类别数目随着聚类过程而变化；
- 对类别数的“合并”： （ 当聚类结果某一类中样本数太少， 或两个类间的距离太近时）（样本太少或太散会被打散）
- “分裂”（ 当聚类结果中某一类的类内方差太大， 将该类进行分裂）

#### Kernel k-means：
kernel k-means实际上公式上，就是将每个样本进行一个投射到高维空间的处理，然后再将处理后的数据使用普通的k-means算法思想进行聚类。

#### 二分K-means：
首先将所有点作为一个簇，然后将该簇一分为二。之后选择能最大限度降低聚类代价函数（也就是误差平方和）的簇划分为两个簇。 以此进行下去，直到簇的数目等于用户给定的数目k为止。

#### Mini Batch K-Means
对大数据分小批量处理

---
学习参考：
- 聚类算法：https://www.jianshu.com/p/2fa67f9bad60

In [1]:
# 使用Python实现kmeans方法
import numpy as np
import random
import logging as log

In [2]:
log.basicConfig(format="%(message)s", level=log.INFO)

def load_data(file_path):
    # 源数据格式为多行，每行为两个浮点数，分别表示(x,y)
    data = []
    with open(file_path, 'r', encoding='utf-8') as fr:
        for line in fr.read().splitlines():
            line_float = list(map(float, line.split('\t')))
            data.append(line_float)
    data = np.array(data)
    return data

In [3]:
# 计算两个点之间的欧式距离
def score_euclidean(a, b):
    s = np.sqrt(np.sum(np.power(a-b, 2)))
    return s

# 随机采样k个样本作为聚类中心
def rand_center(data, k):
    centers = np.array(random.sample(list(data), k))
    return centers

In [9]:
def k_means(data, k, max_iter=100, score=score_euclidean, e=1e-6):
    """
    K-Means 算法
    一般 K-Mean 算法的终止条件有如下几个：
        1. 所有样本的类别不再改变
        2. 达到最大迭代次数
        3. 精度达到要求（？）

    返回聚类中心及聚类结果
    """
    
    n = len(data)
    
    # 每个结果为一个二元组 [label, score] 分别保存每个样本所在的簇及距离质心的距离
    ret = np.array([[-1, np.inf]] * n)
    
    # 选取聚类中心
    centers = rand_center(data, k)
    
    changed = True    # 标记样本类别是否改变
    n_iter = 0        # 记录迭代次数
    while changed and n_iter < max_iter:
        changed = False
        n_iter += 1
        
        for i in range(n):      # 对每个数据
            i_score = np.inf
            i_label = -1
            for j in range(k):    # 与每个质点比较
                s_ij = score(data[i], centers[j])
                if s_ij < i_score:
                    i_score = s_ij
                    i_label = j
            
            if ret[i, 0] != i_score:      # 样本的类别发生了改变
                changed = True
                
            ret[i, :] = i_label, i_score
            
        # 更新聚类中心
        log.info(centers)
        for i in range(k):
            data_i = data[ret[:, 0] == i]    # 标签为i的样本
            centers[i, :] = np.mean(data_i, axis=0)    # 按类别过滤样本
   
    log.info(n_iter)    # 迭代次数
    return centers, ret

In [10]:
def _test():
    """"""
    file_path = "./data_sets/data.txt"

    data = load_data(file_path)
    print(data)
    print(np.shape(data)[1])

    s = score_euclidean(data[0], data[1])
    print(s)

    centers = rand_center(data, 3)
    print(centers)

In [11]:
if __name__ == '__main__':
    """"""
    # _test()

    file_path = "./data_sets/data.txt"
    data = load_data(file_path)

    centers, ret = k_means(data, 3)
    # print(ret)

[[-0.790153  3.343144]
 [-3.156485  3.191137]
 [ 1.889034  5.1904  ]]
[[ 1.38241133 -1.20847963]
 [-3.32446816 -0.63445887]
 [ 2.89375324  2.72741571]]
[[ 2.65077367 -2.79019029]
 [-3.14333833 -0.25014678]
 [ 2.13934243  3.12797287]]
[[ 2.65077367 -2.79019029]
 [-3.18695357 -0.35938491]
 [ 1.98283629  3.1465235 ]]
[[ 2.65077367 -2.79019029]
 [-3.18695357 -0.35938491]
 [ 1.98283629  3.1465235 ]]
[[ 2.65077367 -2.79019029]
 [-3.18695357 -0.35938491]
 [ 1.98283629  3.1465235 ]]
[[ 2.65077367 -2.79019029]
 [-3.18695357 -0.35938491]
 [ 1.98283629  3.1465235 ]]
[[ 2.65077367 -2.79019029]
 [-3.18695357 -0.35938491]
 [ 1.98283629  3.1465235 ]]
[[ 2.65077367 -2.79019029]
 [-3.18695357 -0.35938491]
 [ 1.98283629  3.1465235 ]]
[[ 2.65077367 -2.79019029]
 [-3.18695357 -0.35938491]
 [ 1.98283629  3.1465235 ]]
[[ 2.65077367 -2.79019029]
 [-3.18695357 -0.35938491]
 [ 1.98283629  3.1465235 ]]
[[ 2.65077367 -2.79019029]
 [-3.18695357 -0.35938491]
 [ 1.98283629  3.1465235 ]]
[[ 2.65077367 -2.79019029]
 